In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
 cd 'drive/MyDrive/A/Practical-Data-Science-with-Python-main/Chapter3/test_your_knowledge'

/content/drive/MyDrive/A/Practical-Data-Science-with-Python-main/Chapter3/test_your_knowledge


In [5]:
!ls

bitcoin_data.sql    chinook_data.csv   chinook.db
bitcoin_price.json  chinook_data.xlsx  Solution.ipynb


In [6]:
import os 
os.remove('bitcoin_data.sql')

In [7]:
import json

with open('bitcoin_price.json') as f:
  data = json.load(f)

In [10]:
data[1]

{'close': 103.999,
 'high': 105.9,
 'low': 92.49999,
 'open': 93.25,
 'symbol': 'btcusd',
 'time': 1364774400000,
 'volume': 5224.40131297}

In [11]:
data_list = []
for d in data:
  data_list.append([d['time'], d['open'], d['close'], d['high'], d['low'],d['volume']])
len(data_list)

2793

In [13]:
import sqlite3

db = sqlite3.connect('bitcoin_data.sql')
cur = db.cursor()

cur.execute('''CREATE TABLE bitcoin_price_n (TIME INTEGER, OPEN REAL, CLOSE REAL, HIGH REAL, LOW REAL, VOLUME REAL);''' )


In [14]:
cur.executemany('INSERT INTO bitcoin_price_n VALUES(?,?,?,?,?,?)',data_list)
db.commit()
db.close()

In [17]:
db = sqlite3.connect('bitcoin_data.sql')
cur = db.cursor()

In [18]:
res = cur.execute('SELECT MAX(time) FROM bitcoin_price_n;')
maxtime = res.fetchone()

In [23]:
import datetime
datetime.datetime.fromtimestamp(maxtime[0] // 1000)

datetime.datetime(2020, 11, 28, 0, 0)

In [25]:
res = cur.execute('SELECT MIN(time) FROM bitcoin_price_n;')
mintime = res.fetchone()
datetime.datetime.fromtimestamp(mintime[0] // 1000)

datetime.datetime(2013, 3, 31, 0, 0)

In [27]:
query = """
SELECT MAX(close), STRFTIME('%Y', DATE(ROUND(time / 1000), 'unixepoch')) as year
FROM bitcoin_price_n
GROUP BY year
ORDER BY year;
"""
res = cur.execute(query)
res.fetchall()

[(1138.5, '2013'),
 (919.24, '2014'),
 (466.01, '2015'),
 (981.7, '2016'),
 (19210.0, '2017'),
 (17169.0, '2018'),
 (12876.0, '2019'),
 (19131.0, '2020')]

In [28]:
db.close()

In [29]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///../chinook.db")
conn = engine.connect()

In [30]:
query = """
SELECT AVG(tracks.Milliseconds) / 1000, genres.Name
FROM tracks
JOIN genres
ON tracks.genreid = genres.genreid
GROUP BY tracks.genreid
ORDER BY AVG(Tracks.Milliseconds) DESC;
"""
res = conn.execute(query)
list(res)

[(2911.7830384615386, 'Sci Fi & Fantasy'),
 (2625.549076923077, 'Science Fiction'),
 (2575.28378125, 'Drama'),
 (2145.041021505376, 'TV Shows'),
 (1585.263705882353, 'Comedy'),
 (309.74944385026737, 'Metal'),
 (302.9858, 'Electronica/Dance'),
 (297.4529285714286, 'Heavy Metal'),
 (293.86756756756756, 'Classical'),
 (291.75537692307694, 'Jazz'),
 (283.9100431765613, 'Rock'),
 (270.35977777777777, 'Blues'),
 (264.05852500000003, 'Alternative'),
 (247.17775862068964, 'Reggae'),
 (244.37088372093024, 'Soundtrack'),
 (234.35384939759035, 'Alternative & Punk'),
 (232.85926252158893, 'Latin'),
 (229.03410416666665, 'Pop'),
 (224.92382142857142, 'World'),
 (220.0668524590164, 'R&B/Soul'),
 (219.59, 'Bossa Nova'),
 (189.16420833333333, 'Easy Listening'),
 (178.1762857142857, 'Hip Hop/Rap'),
 (174.813, 'Opera'),
 (134.6435, 'Rock And Roll')]

In [31]:
query = """
SELECT SUM(invoices.total), STRFTIME('%Y', invoices.InvoiceDate) as year, genres.name
FROM invoices
JOIN invoice_items
ON invoices.invoiceid = invoice_items.invoiceid
JOIN tracks
ON invoice_items.trackid = tracks.trackid
JOIN genres
ON tracks.genreid = genres.genreid
GROUP BY year, tracks.genreid
ORDER BY year;
"""
res = conn.execute(query)
res_list = list(res)
res_list

[(1561.2300000000005, '2009', 'Rock'),
 (185.13, '2009', 'Jazz'),
 (502.92000000000013, '2009', 'Metal'),
 (466.29000000000025, '2009', 'Alternative & Punk'),
 (13.86, '2009', 'Rock And Roll'),
 (112.85999999999999, '2009', 'Blues'),
 (789.0300000000002, '2009', 'Latin'),
 (58.41, '2009', 'Reggae'),
 (13.86, '2009', 'Pop'),
 (45.53999999999999, '2009', 'Soundtrack'),
 (13.86, '2009', 'Bossa Nova'),
 (41.58, '2009', 'Easy Listening'),
 (50.489999999999995, '2009', 'Heavy Metal'),
 (69.3, '2009', 'R&B/Soul'),
 (17.82, '2009', 'Electronica/Dance'),
 (41.58, '2009', 'World'),
 (27.72, '2009', 'Hip Hop/Rap'),
 (1364.7700000000013, '2010', 'Rock'),
 (173.24999999999997, '2010', 'Jazz'),
 (488.2300000000002, '2010', 'Metal'),
 (264.37999999999994, '2010', 'Alternative & Punk'),
 (27.72, '2010', 'Rock And Roll'),
 (44.550000000000004, '2010', 'Blues'),
 (804.1400000000006, '2010', 'Latin'),
 (58.41, '2010', 'Reggae'),
 (61.38, '2010', 'Pop'),
 (55.44, '2010', 'Soundtrack'),
 (27.72, '2010', 'B

In [33]:
import pandas as pd
df = pd.DataFrame(res_list, columns=['sales_USD', 'year', 'genre'])
df.to_csv('chinook_data.csv', index=False)
df.to_excel('chinook_data.xlsx', index=False)